In [ ]:
import time
from typing import Dict

import cv2
import numpy as np

from pystream import Pipeline, Stage

In [ ]:
class DummyStage(Stage):
    """A dummy stage that performs some convolutions to the input 2D array,
    and count how many input it has processed.
    
    It is recommended to define 'name' property in the stage instance init
    """

    def __init__(self, name: str) -> None:
        """Initialize stage

        Args:
            name (str): stage name
        """        
        self.count = 0
        self.name = name
        self.kernel = np.random.randint(-10, 10, size=(5, 5))

    def __call__(self, data: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
        """Main data processing of the stage, a mandatory method of the Stage 
        class. Note that the stage does not return a new dict object, but only
        modifies the input object and returns it as the output

        Args:
            data (Dict[str, np.ndarray]): the input array, packed
                in a dictionary

        Returns:
            Dict[str, np.ndarray]: the output array, packed
            in a dictionary
        """
        img = data["data"]
        for _ in range(100):
            img = cv2.filter2D(src=img, ddepth=-1, kernel=self.kernel)
        data["data"] = img
        self.count += 1
        return data

    def cleanup(self):
        """Cleanup method, called at the end of the pipeline.
        in this example, we want to reset the counter to 0"""
        self.count = 0

In [ ]:
def generate_data() -> Dict[str, np.ndarray]:
    img = np.random.randint(0, 255, size=(480, 720, 3), dtype=np.uint8)
    return {"data": img}

def create_pipeline() -> Pipeline:
    # First, create the pipeline instance, we want to use the profiler
    pipeline = Pipeline(input_generator=generate_data, use_profiler=True)
    # Now, add 5 dummy stages to the pipeline.
    pipeline.add(DummyStage("Stage1"))
    pipeline.add(DummyStage("Stage2"))
    pipeline.add(DummyStage("Stage3"))
    pipeline.add(DummyStage("Stage4"))
    pipeline.add(DummyStage("Stage5"))
    return pipeline

In [ ]:
pipeline = create_pipeline()
print("Starting pipeline in serial...")
pipeline.serialize()
pipeline.start_loop(INPUT_PERIOD)
time.sleep(ON_TIME)
print("Stopping pipeline...")
pipeline.stop_loop()

# Let's try read the last result
latest = pipeline.get_results()
print("Last output:")
print(latest)
pipeline.cleanup()